In [2]:
import pandas as pd
import time
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch
from torch.utils.data import random_split

In [3]:
data_df = pd.read_pickle('data_df_2015-2020.pickle')


In [4]:
games_df = pd.read_pickle('games_df_2015-2020.pickle')


In [5]:
training_data = []
for i in range(len(games_df)):
    try:
        game = games_df.loc[i]
        homeTeam, awayTeam = str(game.Home), str(game.Away)
        date_str = str(game.Date.date())
        home_index = (date_str, homeTeam) # Index to get the stats for a team on a certain date from data_df
        away_index = (date_str, awayTeam)
        home_stats = data_df.loc[home_index].to_numpy()
        away_stats = data_df.loc[away_index].to_numpy()
        # array_to_append = np.array([ np.array([home_stats.to_numpy(), away_stats.to_numpy()]), game.DidHomeWin])
        # stats = np.array([home_stats.to_numpy(), away_stats.to_numpy()]) # make a 2d array for model eventually
        training_data.append([home_stats, away_stats, int(game.DidHomeWin)]) # [1, 0 ] if AWAY WON,and [0,1] if HOME WON
    except KeyError as e:
        print(date_str) 
        print(home_index)

)
2019-12-25
('2019-12-25', 'Philadelphia')
2019-12-25
('2019-12-25', 'Golden State')
2019-12-25
('2019-12-25', 'LA Lakers')
2019-12-25
('2019-12-25', 'Denver')
2019-12-26
('2019-12-26', 'Detroit')
2019-12-26
('2019-12-26', 'Brooklyn')
2019-12-26
('2019-12-26', 'Dallas')
2019-12-26
('2019-12-26', 'Okla City')
2019-12-26
('2019-12-26', 'Sacramento')
2019-12-26
('2019-12-26', 'Utah')
2019-12-27
('2019-12-27', 'Boston')
2019-12-27
('2019-12-27', 'Charlotte')
2019-12-27
('2019-12-27', 'Orlando')
2019-12-27
('2019-12-27', 'Atlanta')
2019-12-27
('2019-12-27', 'Miami')
2019-12-27
('2019-12-27', 'Golden State')
2019-12-28
('2019-12-28', 'Denver')
2019-12-28
('2019-12-28', 'Boston')
2019-12-28
('2019-12-28', 'New Orleans')
2019-12-28
('2019-12-28', 'Chicago')
2019-12-28
('2019-12-28', 'Houston')
2019-12-28
('2019-12-28', 'Miami')
2019-12-28
('2019-12-28', 'Minnesota')
2019-12-28
('2019-12-28', 'Washington')
2019-12-28
('2019-12-28', 'Golden State')
2019-12-28
('2019-12-28', 'Milwaukee')
2019-12

In [6]:
training_data[0]

[array([107.2  ,   0.914,  13.3  ,  28.3  ,   1.039,  40.2  ,  12.7  ,
         75.   ,  31.   ,  69.   ,  25.   ]),
 array([114.4  ,   1.042,   8.7  ,  27.7  ,   1.052,  44.3  ,   1.   ,
         78.2  ,  22.1  ,  77.9  ,  21.8  ]),
 1]

In [7]:
num_data_points = len(training_data[0][0])*2
x_vals = [x[0:2] for x in training_data]
x_vals = torch.FloatTensor(x_vals)
x_vals = x_vals.view(-1, num_data_points)
y_vals = [y[2] for y in training_data]
y_vals = torch.LongTensor(y_vals)
print(x_vals[0], y_vals[0])
x_vals.shape

tensor([107.2000,   0.9140,  13.3000,  28.3000,   1.0390,  40.2000,  12.7000,
         75.0000,  31.0000,  69.0000,  25.0000, 114.4000,   1.0420,   8.7000,
         27.7000,   1.0520,  44.3000,   1.0000,  78.2000,  22.1000,  77.9000,
         21.8000]) tensor(1)


torch.Size([5342, 22])

In [8]:
dataset = TensorDataset(x_vals, y_vals)

In [9]:
len(dataset) - 500

4842

In [10]:
train_ds, val_ds = random_split(dataset, [4842, 500])

In [11]:
batch_size = 32
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [12]:
import torch.nn as nn
import torch.nn.functional as F

In [13]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(22, 32) 
        self.fc2 = nn.Linear(32,2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [14]:
for data, win in train_loader:
    print(data)
    print(win)
    net = Net()
    outputs = net(data)
    print(outputs)
    break

tensor([[ 99.8000,   1.0410,   0.7000,  23.7000,   0.9900,  43.6000,  -5.7000,
          70.6000,  25.5000,  74.5000,  29.4000, 106.8000,   1.0470,   2.7000,
          28.3000,   1.0570,  48.2000,   1.0000,  71.6000,  26.2000,  73.8000,
          28.4000],
        [106.3000,   1.0280,   0.7000,  21.7000,   1.0310,  41.3000,   0.3000,
          76.8000,  28.0000,  72.0000,  23.2000, 109.3000,   1.0340,   1.0000,
          28.0000,   1.0580,  45.8000,   2.3000,  75.8000,  23.2000,  76.8000,
          24.2000],
        [106.7000,   0.9490,   7.7000,  23.7000,   1.0100,  40.9000,   6.3000,
          77.9000,  17.5000,  82.5000,  22.1000, 117.2000,   1.0540,   7.7000,
          30.7000,   1.1030,  44.6000,   5.0000,  74.0000,  21.5000,  78.5000,
          26.0000],
        [ 98.1000,   1.0100,  -4.3000,  26.3000,   0.9750,  42.4000,  -3.7000,
          75.6000,  21.8000,  78.2000,  24.4000, 107.2000,   1.2190, -16.3000,
          29.3000,   1.0360,  52.5000, -19.0000,  82.4000,  23.5000,  7

In [15]:
probs = F.softmax(outputs, dim=1)
probs
max_probs, preds = torch.max(probs, dim = 1)
print(preds)
torch.sum(preds == win)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


tensor(7)

In [16]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [17]:
loss_fn = F.cross_entropy

In [18]:
loss = loss_fn(outputs, win)
print(loss)

tensor(1.9643, grad_fn=<NllLossBackward>)


In [19]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [30]:
def fit(epochs, learning_rate, model, train_loader, val_loader, opt_func=torch.optim.AdamW): # mess around with optimizer
    optimizer = opt_func(model.parameters(), learning_rate)
    history = [] # for recording epoch-wise results
    
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)po
        history.append(result)
        adjust_learning_rate(learning_rate, optimizer, epoch)

    return history

In [31]:
class BBallNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(22, 64) 
        self.fc2 = nn.Linear(64,2)
        
    def forward(self, data):
        out = self.fc1(data)
        out = self.fc2(out)
        return out
    
    def training_step(self, batch):
        data, win_labels = batch 
        out = self(data)                  # Generate predictions
        loss = F.cross_entropy(out, win_labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate losszz
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [32]:
def adjust_learning_rate(lr, optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [33]:
BBModel = BBallNet()

In [34]:
history1 = fit(30, 0.001, BBModel, train_loader, val_loader)

Epoch [0], val_loss: 0.6878, val_acc: 0.6035
Epoch [1], val_loss: 0.6906, val_acc: 0.5785
Epoch [2], val_loss: 0.7454, val_acc: 0.5195
Epoch [3], val_loss: 0.6751, val_acc: 0.5887
Epoch [4], val_loss: 0.6629, val_acc: 0.5945
Epoch [5], val_loss: 0.6621, val_acc: 0.5809
Epoch [6], val_loss: 0.7055, val_acc: 0.5211
Epoch [7], val_loss: 0.6766, val_acc: 0.5684
Epoch [8], val_loss: 0.6969, val_acc: 0.5617
Epoch [9], val_loss: 0.6681, val_acc: 0.6211
Epoch [10], val_loss: 0.7023, val_acc: 0.5578
Epoch [11], val_loss: 0.6984, val_acc: 0.5551
Epoch [12], val_loss: 0.7368, val_acc: 0.4785
Epoch [13], val_loss: 0.6769, val_acc: 0.5906
Epoch [14], val_loss: 0.6770, val_acc: 0.6062
Epoch [15], val_loss: 0.9150, val_acc: 0.4148
Epoch [16], val_loss: 0.8155, val_acc: 0.4531
Epoch [17], val_loss: 0.7677, val_acc: 0.6125
Epoch [18], val_loss: 0.7201, val_acc: 0.5223
Epoch [19], val_loss: 0.7290, val_acc: 0.5023
Epoch [20], val_loss: 0.6747, val_acc: 0.5750
Epoch [21], val_loss: 0.6755, val_acc: 0.578

In [43]:
def determine_pythag_expectation(points_against, points_for):
    win = (points_for**(13.91)) / (points_for**13.91 + points_against**13.91)
    return win
def get_point_differential(points_against, points_for):
    return points_for-points_against

In [ ]:
correct = 0
extremly_wrong = 0
for data in training_data:
    home_expectation = determine_pythag_expectation(data[0][0], data[0][1])
    away_expectation = determine_pythag_expectation(data[1][0], data[1][1])
    #print(f"Home: {home_expectation} Away :{away_expectation} HomeWin: {data[2]}")
    if (data[0][1] >= data[1][1]) and data[2]:
        correct += 1
    else:
        extremly_wrong += 1

print(correct)
print(extremly_wrong)
print(correct/(correct+extremly_wrong))